# Fairness on COMPAS using Inferred Attributes and autogluon


We demonstrate how to enforce a wide range of fairness definitions on the COMPAS dataset. This dataset records paroles caught violating the terms of parole. As it measures who was caught, it is strongly influenced by policing and environmental biases, and should not be confused with a measurement of who actually violated their terms of parole. See [this paper](https://datasets-benchmarks-proceedings.neurips.cc/paper/2021/file/92cc227532d17e56e07902b254dfad10-Paper-round1.pdf) for a discussion of its limitations and caveats.

We use it because it is a standard fairness dataset that captures such strong differences in outcome between people identified as African-American and everyone else, that classifiers trained on this dataset violate most definitions of fairness.

As many of the ethnic groups are too small for reliable statistical estimation, we only consider differences is in outcomes between African-Americans vs. everyone else (labeled as other).
We load and preprocess the COMPAS dataset, splitting it into three roughly equal partitions of train, validation, and test:


In [1]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
from oxonfair import FairPredictor, inferred_attribute_builder
from oxonfair.utils import group_metrics as gm
all_data = pd.read_csv('https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv')
condensed_data=all_data[['sex','race','age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'age_cat', 'c_charge_degree','two_year_recid']].copy()
condensed_data.replace({'Caucasian':'Other', 'Hispanic':'Other', 'Native American':'Other', 'Asian':'Other'},inplace=True)
train=condensed_data.sample(frac=0.3, random_state=0)
val_and_test=condensed_data.drop(train.index)
val=val_and_test.sample(frac=0.5, random_state=0)
test=val_and_test.drop(val.index)

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


To enforce fairness constraints without access to protected attributes at test time, we train two classifiers to infer the 2-year recidivism rate, and ethnicity.

In [2]:
predictor2, protected = inferred_attribute_builder(train, 'two_year_recid', 'race', time_limit=5)

No path specified. Models will be saved in: "AutogluonModels/ag-20240611_071651"


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.


Beginning AutoGluon training ... Time limit = 5s


AutoGluon will save models to "AutogluonModels/ag-20240611_071651"


=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.5.0: Wed May  1 20:14:38 PDT 2024; root:xnu-10063.121.3~5/RELEASE_ARM64_T6020
CPU Count:          10
Memory Avail:       5.06 GB / 16.00 GB (31.6%)
Disk Space Avail:   371.62 GB / 460.43 GB (80.7%)


...

	0.6582	 = Validation score   (accuracy)


	0.03s	 = Training   runtime


	0.0s	 = Validation runtime


AutoGluon training complete, total runtime = 5.47s ... Best model: "WeightedEnsemble_L2"


TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240611_071658")


From these a single predictor that maximizes accuracy while reducing the demographic parity violation to less than 2.5% can be trained by running:

In [3]:
fpredictor=FairPredictor(predictor2, val, 'race', inferred_groups=protected)
fpredictor.fit(gm.accuracy, gm.demographic_parity, 0.025)

Now we will show how a family of fairness measures can be individually optimized. First, we consider the measures of Sagemaker Clarify. 

The following code plots a table showing the change in accuracy and the fairness measure on a held-out test set as we decrease the fairness measure to less than 0.025 (on validation) for all measures except for disparate impact which we raise to above 0.975.
We define a helper function for evaluation:

In [4]:
def evaluate(fpredictor,use_metrics):
    "Print a table showing the accuracy drop that comes with enforcing fairness"
    extra_metrics = {**use_metrics, 'accuracy':gm.accuracy}
    collect=pd.DataFrame(columns=['Measure (original)', 'Measure (updated)', 'Accuracy (original)', 'Accuracy (updated)'])
    for d in use_metrics.items():
        if d[1].greater_is_better is False:
            fpredictor.fit(gm.accuracy, d[1], 0.025)
        else:
            fpredictor.fit(gm.accuracy, d[1], 1-0.025)
        tmp=fpredictor.evaluate_fairness(test, metrics=extra_metrics,verbose=False)
        collect.loc[d[1].name]=np.concatenate((np.asarray(tmp.loc[d[0]]), np.asarray(tmp.loc['accuracy'])), 0)
    print(collect.to_markdown())

We can now contrast the behavior of a fair classifier that relies on access to the protected attribute at test time with one that infers it.

In [5]:
# we first create a classifier using the protected attribute
predictor=TabularPredictor(label='two_year_recid').fit(train_data=train,time_limit=5)
fpredictor = FairPredictor(predictor, val, 'race', )
evaluate(fpredictor, gm.clarify_metrics)


No path specified. Models will be saved in: "AutogluonModels/ag-20240611_071704"


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.


Beginning AutoGluon training ... Time limit = 5s


AutoGluon will save models to "AutogluonModels/ag-20240611_071704"


=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.5.0: Wed May  1 20:14:38 PDT 2024; root:xnu-10063.121.3~5/RELEASE_ARM64_T6020
CPU Count:          10
Memory Avail:       2.93 GB / 16.00 GB (18.3%)
Disk Space Avail:   371.58 GB / 460.43 GB (80.7%)


...

	0.03s	 = Training   runtime


	0.0s	 = Validation runtime


AutoGluon training complete, total runtime = 5.37s ... Best model: "WeightedEnsemble_L2"


TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240611_071704")


|                                                         |   Measure (original) |   Measure (updated) |   Accuracy (original) |   Accuracy (updated) |
|:--------------------------------------------------------|---------------------:|--------------------:|----------------------:|---------------------:|
| Demographic Parity                                      |          0.33658     |          0.0102499  |              0.661386 |             0.646337 |
| Disparate Impact                                        |          0.424223    |          0.985274   |              0.661386 |             0.638812 |
| Average Group Difference in Conditional Acceptance Rate |          0.644987    |          0.055467   |              0.661386 |             0.670891 |
| Average Group Difference in Conditional Rejectance Rate |          0.354906    |          0.00356274 |              0.661386 |             0.666535 |
| Average Group Difference in Accuracy                    |          0.0484221   |      

In contrast, even though the base classifiers have similar accuracy, when using inferred attributes (N.B. the base classifier is not directly trained to maximize accuracy, which is why it can have higher accuracy when it doesn't use race), we see a much greater drop in accuracy as fairness is enforced which is consistent with [Lipton et al.](https://arxiv.org/pdf/1711.07076.pdf)


In [6]:
# Now using the inferred attributes
fpredictor2 = FairPredictor(predictor2, val, 'race', inferred_groups=protected)
evaluate(fpredictor2, gm.clarify_metrics)

|                                                         |   Measure (original) |   Measure (updated) |   Accuracy (original) |   Accuracy (updated) |
|:--------------------------------------------------------|---------------------:|--------------------:|----------------------:|---------------------:|
| Demographic Parity                                      |            0.224384  |           0.0429492 |               0.65901 |             0.56     |
| Disparate Impact                                        |            0.568389  |           0.933854  |               0.65901 |             0.553267 |
| Average Group Difference in Conditional Acceptance Rate |            0.293743  |           0.0034529 |               0.65901 |             0.596832 |
| Average Group Difference in Conditional Rejectance Rate |            0.139117  |           0.0473963 |               0.65901 |             0.653069 |
| Average Group Difference in Accuracy                    |            0.0641907 |      

Similar results can be obtained using the metrics of Verma and Rubin, by running

In [7]:
evaluate(fpredictor, gm.verma_metrics)


|                                                 |   Measure (original) |   Measure (updated) |   Accuracy (original) |   Accuracy (updated) |
|:------------------------------------------------|---------------------:|--------------------:|----------------------:|---------------------:|
| Statistical Parity                              |          0.33658     |           0.0102499 |              0.661386 |             0.646337 |
| Predictive Parity                               |          0.000623377 |           0.0172127 |              0.661386 |             0.666139 |
| Equal Opportunity                               |          0.306877    |           0.0652084 |              0.661386 |             0.640396 |
| Average Group Difference in False Negative Rate |          0.306877    |           0.0652084 |              0.661386 |             0.640396 |
| Equalized Odds                                  |          0.301797    |           0.0154928 |              0.661386 |             0.6

And again for infered attributes. 

In [8]:
evaluate(fpredictor2, gm.verma_metrics)


No solutions satisfy the constraint found, selecting the
                           closest solution


|                                                 |   Measure (original) |   Measure (updated) |   Accuracy (original) |   Accuracy (updated) |
|:------------------------------------------------|---------------------:|--------------------:|----------------------:|---------------------:|
| Statistical Parity                              |            0.224384  |           0.0429492 |               0.65901 |             0.56     |
| Predictive Parity                               |            0.0166061 |           0.15674   |               0.65901 |             0.565545 |
| Equal Opportunity                               |            0.162329  |           0.0118372 |               0.65901 |             0.546931 |
| Average Group Difference in False Negative Rate |            0.162329  |           0.0118372 |               0.65901 |             0.546931 |
| Equalized Odds                                  |            0.185081  |           0.0693341 |               0.65901 |             0.5